In [1]:
import numpy as np
import pandas as pd

In [2]:
from google.colab import files
path=list(files.upload().keys())[0]

Saving fashion_products.csv to fashion_products.csv


In [3]:
df=pd.read_csv(path)

In [4]:
df

,User ID,Product ID,Product Name,Brand,Category,Price,Rating,Color,Size
0,19,1,Dress,Adidas,Men's Fashion,40,1.043159,Black,XL
1,97,2,Shoes,H&M,Women's Fashion,82,4.026416,Black,L
2,25,3,Dress,Adidas,Women's Fashion,44,3.337938,Yellow,XL
3,57,4,Shoes,Zara,Men's Fashion,23,1.049523,White,S
4,79,5,T-shirt,Adidas,Men's Fashion,79,4.302773,Black,M
...,...,...,...,...,...,...,...,...,...
995,20,996,Shoes,Zara,Women's Fashion,55,1.620081,Black,M
996,42,997,Sweater,Nike,Kids' Fashion,13,1.544464,Green,L
997,9,998,Sweater,Zara,Men's Fashion,47,3.961913,White,L
998,8,999,T-shirt,Zara,Women's Fashion,68,3.089722,Blue,S


In [5]:
!pip install scikit-surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.0/772.0 kB 11.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.3-cp310-cp310-linux_x86_64.whl size=2811603 sha256=4bb8921a6ea74d5b639fc476f483e3767baa5c1db33aa4d078d44331deebb029
  Stored in directory: /root/.cache/pip/wheels/a5/ca/a8/4e28def53797fdc4363ca4af740db15a9c2f1595ebc51fb445
Successfully built scikit-surprise


In [7]:
from surprise import Dataset , Reader , SVD
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

In [9]:
new_df = df[['Product ID', 'Product Name', 'Brand',
                   'Category', 'Color', 'Size']]
new_df["Content"]=new_df.apply(lambda x : " ".join(x.dropna().astype(str)),axis=1)

<ipython-input-9-f66b33ea1661>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df["Content"]=new_df.apply(lambda x : " ".join(x.dropna().astype(str)),axis=1)


In [10]:
new_df

,Product ID,Product Name,Brand,Category,Color,Size,Content
0,1,Dress,Adidas,Men's Fashion,Black,XL,1 Dress Adidas Men's Fashion Black XL
1,2,Shoes,H&M,Women's Fashion,Black,L,2 Shoes H&M Women's Fashion Black L
2,3,Dress,Adidas,Women's Fashion,Yellow,XL,3 Dress Adidas Women's Fashion Yellow XL
3,4,Shoes,Zara,Men's Fashion,White,S,4 Shoes Zara Men's Fashion White S
4,5,T-shirt,Adidas,Men's Fashion,Black,M,5 T-shirt Adidas Men's Fashion Black M
...,...,...,...,...,...,...,...
995,996,Shoes,Zara,Women's Fashion,Black,M,996 Shoes Zara Women's Fashion Black M
996,997,Sweater,Nike,Kids' Fashion,Green,L,997 Sweater Nike Kids' Fashion Green L
997,998,Sweater,Zara,Men's Fashion,White,L,998 Sweater Zara Men's Fashion White L
998,999,T-shirt,Zara,Women's Fashion,Blue,S,999 T-shirt Zara Women's Fashion Blue S


In [12]:
tfidf=TfidfVectorizer()
metrics=tfidf.fit_transform(new_df["Content"]).toarray()

In [14]:
metrics.shape

(1000, 1011)

In [15]:
similarity=linear_kernel(metrics,metrics)

In [81]:
def content_based( Product_ID , top_n):
  index=new_df[new_df["Product ID"]== Product_ID ].index[0]
  index_similarity=[i[0] for i in sorted(list(enumerate(similarity[5])),reverse=True,key = lambda x: x[1])[1:6]]
  print(index_similarity)
  return new_df.iloc[index_similarity, : 5]

In [82]:
content_based(1,5)

[2, 0, 117, 398, 221]


,Product ID,Product Name,Brand,Category,Color
2,3,Dress,Adidas,Women's Fashion,Yellow
0,1,Dress,Adidas,Men's Fashion,Black
117,118,Dress,Adidas,Men's Fashion,Yellow
398,399,Dress,Adidas,Men's Fashion,Yellow
221,222,Dress,Adidas,Kids' Fashion,Yellow


In [41]:
reader=Reader(rating_scale=(1,5))
data=Dataset.load_from_df(df[['User ID','Product ID','Rating']], reader)

In [43]:
svd=SVD()
training=data.build_full_trainset()
svd.fit(training)

In [77]:
def collabrative_filtering( Product_ID , top_n):
    testset = training.build_anti_testset()
    testset = filter(lambda x: x[0] == Product_ID , testset)
    predictions = svd.test(testset)
    predictions.sort(key=lambda x: x.est, reverse=True)
    recommendations = [prediction.iid for prediction in predictions[:top_n]]
    print(recommendations)
    return new_df.iloc[recommendations , :-2]



In [78]:
collabrative_filtering(1,5)


[822, 82, 605, 642, 63]


,Product ID,Product Name,Brand,Category,Color
822,823,T-shirt,H&M,Kids' Fashion,White
82,83,Shoes,Nike,Women's Fashion,Blue
605,606,Dress,Nike,Women's Fashion,White
642,643,T-shirt,Zara,Kids' Fashion,Black
63,64,Shoes,Gucci,Kids' Fashion,Red


In [73]:
def hybrid_recommandation(Product_ID , top_n):
  content_value=content_based( Product_ID , top_n)
  collabrative_based=collabrative_filtering( Product_ID , top_n)
  total_index=list(set(content_value + collabrative_based ))
  return new_df.iloc[total_index , : -2]

In [74]:
hybrid_recommandation(5,5)

,Product ID,Product Name,Brand,Category,Color
0,1,Dress,Adidas,Men's Fashion,Black
2,3,Dress,Adidas,Women's Fashion,Yellow
549,550,T-shirt,Nike,Men's Fashion,Yellow
390,391,Dress,Gucci,Women's Fashion,Blue
829,830,Jeans,Gucci,Kids' Fashion,Green
398,399,Dress,Adidas,Men's Fashion,Yellow
117,118,Dress,Adidas,Men's Fashion,Yellow
889,890,T-shirt,H&M,Men's Fashion,Black
221,222,Dress,Adidas,Kids' Fashion,Yellow
991,992,T-shirt,Gucci,Women's Fashion,Black
